In [ ]:
Notebook rápido MNIST — versão para minicurso. Cobre: carregar dataset via tensorflow_datasets, normalizar, treinar uma MLP simples e salvar o modelo.

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


Cell 1 — Checagem de ambiente:
Verifica versão do TensorFlow e disponibilidade de GPU. Não instala pacotes no Colab para evitar conflitos.

In [ ]:
import tensorflow as tf
import sys
print('Python', sys.version.split()[0])
print('TensorFlow', tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print('GPUs found:', len(gpus))

Cell 2 — Carregar MNIST com tensorflow_datasets:
Usamos `tfds.load` para baixar e preparar os conjuntos de treino e teste.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load('mnist', split=['train','test'], shuffle_files=True, as_supervised=True, with_info=True)

Cell 3 — Normalizar e preparar o pipeline tf.data:
Transformações: cast/normalize, cache, shuffle e batch. Prefetch para throughput.

In [ ]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(1000).batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(128).prefetch(tf.data.AUTOTUNE)

Cell 4 — Definir e compilar modelo:
Modelo simples: Flatten -> Dense 128 (ReLU) -> Dense 10 (logits). Usamos SparseCategoricalCrossentropy com from_logits=True.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28,1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

Cell 5 — Treinar e salvar:
Treina por algumas épocas (rápido para minicurso) e salva o modelo no formato Keras.

In [ ]:
model.fit(ds_train, epochs=3, validation_data=ds_test)
model.save('mnist_model.keras')
print('Modelo salvo em mnist_model.keras')